In [1]:
import sympy,time,pyperclip
from sympy.matrices import Matrix #最后处理矩阵用
#导入工具包
load('pkg/Lie.sage') #李代数相关的工具
load('pkg/spo.sage') #李超代数spo的工具
load('pkg/MatSp.sage') #处理矩阵的工具
load('pkg/AlgebraBSC.sage') #结构常数相关工具
load('pkg/pbw.sage') #李代数pbw基的工具
load('pkg/pbws.sage') #李超代数pbw基的工具
load('pkg/rex.py') #解方程等相关工具
print('''导入相关工具包：
1. Lie.sage：李代数工具，以及C,D族李代数
2. spo.sage：李超代数工具，以及spo
3. MatSp.sage：矩阵空间，求矩阵关于矩阵基的线性表示
4. AlgebraBSC：求李/李超代数的结构常数，以及由结构常数生成代数
5. pbw.sage/pbws.sage 李/李超代数的pbw基
6. rex.py 解方程工具''')

导入相关工具包：
1. Lie.sage：李代数工具，以及C,D族李代数
2. spo.sage：李超代数工具，以及spo
3. MatSp.sage：矩阵空间，求矩阵关于矩阵基的线性表示
4. AlgebraBSC：求李/李超代数的结构常数，以及由结构常数生成代数
5. pbw.sage/pbws.sage 李/李超代数的pbw基
6. rex.py 解方程工具


In [2]:
#V初始化数据
tt = time.time()
m,n = 1,2
L = SpO(m,n) #李超代数spo(2m|2n)
mats = L.negative_matrixs + L.h_matrixs + L.positive_matrixs #矩阵信息
pos_m = len(L.positive_roots)
print('李超代数：spo(%d,%d)\n空间维数为：%d\n正根数目为：%d'%(2*m,2*n,len(mats),pos_m))
es = Lie.symbols('e',pos_m)
fs = Lie.symbols('f',pos_m)
hs = Lie.symbols('h',m+n) 
syms = fs + hs +es #用作结构常数的符号变量
#奇根对应的符号变量
roots = L.positive_roots 
ind1 = [roots.index(r) for r in L.odd_roots if r in roots]
roots = L.simple_roots
ind2 = [roots.index(r) for r in L.odd_roots if r in roots]
odds = [fs[i] for i in ind1] + [hs[i] for i in ind2] + [es[i] for i in ind1] 

#计算结构常数
print('计算结构常数')
N = AlgebraBSC.lie_sup_SC(mats,L.odd_mats)
'''
Lf = {i+1:Element(element,N,syms) for i,element in enumerate(fs)}
Le = {i+1:Element(element,N,syms) for i,element in enumerate(es)}
Lh = {i+1:Element(element,N,syms) for i,element in enumerate(hs)}
#'''
#pbw基
print('计算李超代数泛包络的pbw基')
vf = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(fs)}
ve = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(es)}
vh = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(hs)}
ide = SPBWElement({tuple():1},N,syms,odds) #幺元
vectors = L.vectors
print('初始化完毕，用时 %.3fs'%(time.time()-tt))

李超代数：spo(2,4)
空间维数为：17
正根数目为：7
计算结构常数
计算李超代数泛包络的pbw基
初始化完毕，用时 1.901s


In [3]:
#权的正根分解
tt = time.time()
s,t = 1,1
weight = L.basis[s-1] + L.basis[m+t-1];print('待讨论的权：',weight) #待讨论的权（正交基形式）
root = L.vector2root(weight) #转单根形式
dec = L.positive_dec(root) #正根分解（元组构成）
negative_roots = L.negative_roots #负根
dec_root = []
for line in dec:
    new = []
    for i,element in enumerate(line):
        if element:
            new = new + [negative_roots[i]]*element
    dec_root.append(tuple(new))
dec_vector = [tuple(L.root2vector(r) for r in line) for line in dec_root]
print('权空间维数：',len(dec)) #分解总数目
print('用时：%.3fs'%(time.time()-tt))
#for line in res_vectors:print(line)
#table([[i for i in line if i] for line in dec_vector]) #分解结果

待讨论的权： delta + epsilon1
权空间维数： 4
用时：0.059s


In [4]:
#相应的pbw基
basis = [tuple(fs[negative_roots.index(i)] for i in line) for line in dec_root]
pbw_basis = [ide.tuple2element(i) for i in basis]
print('权空间对应的pbw基：')
#pbw_basis

权空间对应的pbw基：


In [5]:
#最高权向量
x = Lie.symbols('x',n)
z = list(Lie.symbols('z',m))
z[s-1] = x[t-1]+2*n-s-t
la = sum([i*j for i,j in zip(z,L.basis[:m])]) + sum([i*j for i,j in zip(x,L.basis[m:])])
#d_s+e_t: a_s = b_t+2n-s-t
print('最高权：',la)

#泛包络上计算结果res
tt = time.time()
res = [[ve[i]*element for element in pbw_basis] for i in range(2,m+n+1)] #略去奇根
print("计算泛包络上的乘积结果，乘积总数：",(m+n)*len(pbw_basis))
print('用时：%.3fs\n'%(time.time()-tt))

#化简为Verma模的元素
tt = time.time()
res_verma = [[i.to_verma(la,L,m+n) for i in line] for line in res]
print('转化为Verma模上结果，用时：%.3fs\n'%(time.time()-tt))

#转化为矩阵
tt = time.time()
data = []
for line in res_verma:
    mat,_ = ide.linear_rep(line)
    data.append(mat)
data = [mat for mat in data if mat] #去掉空项
mat = matrix.block(len(data),1,data)
mat_sym = Matrix(mat)
print('矩阵阶数为 %d*%d  '%mat.dimensions(),end='')
print('计算用时：%.3fs'%(time.time()-tt))
#save_vari(mat_sym,'mat%d.pydata'%k) #保存数据

最高权： delta*(x1 + 2) + epsilon1*x1 + epsilon2*x2
计算泛包络上的乘积结果，乘积总数： 12
用时：1.404s

转化为Verma模上结果，用时：0.397s

矩阵阶数为 4*4  计算用时：0.008s


In [6]:
tt = time.time()
data = []
for line in res_verma:
    mat,_ = ide.linear_rep(line)
    data.append(mat)
data = [mat for mat in data if mat] #去掉空项
mat = matrix.block(len(data),1,data)
mat_sym = Matrix(mat)
print('矩阵阶数为 %d*%d  '%mat.dimensions(),end='')
print('计算用时：%.3fs'%(time.time()-tt))

矩阵阶数为 4*4  计算用时：0.007s


In [10]:
mat_sym.subs({x1:1,x2:-1})

Matrix([
[0, 3, 1, -1],
[2, 0, 1,  0],
[0, 1, 0,  0],
[0, 1, 1, -1]])

In [19]:
print(symmat_to_matlab(mat_sym,4))

syms x1 x2 x3 x4 
[[0, 2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x1 - x2 + 1, 0, 0, 0, 0, -1, 0, 0, 0]; [0, 0, 2*x1 - 2*x2 + 2, -1, 0, 0, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]; [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, x1 - x2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]; [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x1 - x2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0]; [0, 2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x1 - x2 + 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]; [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x1 -

/opt/sagemath-9.0/local/lib/python3.7/site-packages/pyperclip/__init__.py:503: UserWarning: Pyperclip's support for Cygwin is not perfect, see https://github.com/asweigart/pyperclip/issues/55
  warnings.warn('Pyperclip\'s support for Cygwin is not perfect, see https://github.com/asweigart/pyperclip/issues/55')


In [21]:
[i.dimensions()[0] for i in data]

[30, 20, 18, 18]

In [ ]:
matrix([[1,-1,-1],[x1,x2,x3],[x1,x2,x3]])

In [6]:
mat_list = [[str(mat[i,j]) for i in range(23)] for j in range(15)]

In [8]:
latex_matrix(mat_list,hrows=[5,13,18])

\begin{align*}
\left(\begin{array}{ccccccccccccccccccccccc}
2*x1 + 2 &0 &0 &0 &0 &0 &0 &2*x1 - 2*x2 + 2 &0 &0 &0 &0 &0 &0 &0 &0 &0 &x2 - x3 &0 &0 &0 &0 &x2 + x3\\
0 &2*x1 + 2 &0 &0 &0 &0 &x1 - x2 &-1 &0 &0 &0 &0 &0 &0 &x2 - x3 + 1 &0 &0 &0 &0 &0 &0 &1 &1\\
1 &1 &1 &0 &1 &-1 &0 &1 &0 &0 &-1 &x1 - x2 + 2 &-1 &x2 - x3 + 1 &0 &0 &0 &0 &x2 + x3 + 1 &0 &0 &0 &0\\
0 &0 &2*x1 + 2 &0 &0 &0 &0 &-1 &0 &x1 - x2 &0 &0 &0 &0 &1 &0 &0 &1 &0 &0 &0 &x2 + x3 + 1 &0\\
0 &1 &0 &0 &0 &0 &0 &0 &0 &0 &x1 - x2 + 1 &0 &0 &0 &0 &x2 - x3 + 1 &0 &0 &1 &0 &0 &0 &0\\\hline
0 &0 &0 &2*x1 + 2 &0 &0 &-1 &0 &-1 &-1 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &1 &0\\
0 &0 &0 &0 &2*x1 + 2 &0 &0 &0 &x1 - x2 &0 &0 &0 &0 &0 &-1 &0 &0 &0 &0 &0 &0 &-1 &0\\
0 &1 &0 &1 &0 &0 &1 &0 &0 &0 &1 &-1 &0 &0 &0 &0 &x2 - x3 + 2 &0 &1 &1 &0 &0 &0\\
0 &0 &1 &1 &0 &1 &0 &0 &0 &1 &0 &-1 &0 &1 &0 &0 &1 &0 &0 &x2 + x3 + 2 &0 &0 &0\\
0 &0 &1 &0 &0 &x1 - x2 + 1 &0 &0 &0 &0 &0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &x2 + x3 + 1 &0 &0\\
0 &0 &0 &1 &0 &0 &0 &0 &0 &0 &-1 

/opt/sagemath-9.0/local/lib/python3.7/site-packages/pyperclip/__init__.py:503: UserWarning: Pyperclip's support for Cygwin is not perfect, see https://github.com/asweigart/pyperclip/issues/55
  warnings.warn('Pyperclip\'s support for Cygwin is not perfect, see https://github.com/asweigart/pyperclip/issues/55')


In [7]:
def latex_matrix(mat,txt=False,hrows=False,hcols=False):
    '''矩阵的latex代码，默认为align环境，不带行线，不带列线'''
    import pyperclip #用于剪贴板
    #确保为字符串
    mat = [[str(i) for i in line] for line in mat] #转字符串
    m,n = len(mat),len(mat[0]) #行列
    #设置列线行线
    if not hrows:
        hrows = {}
    if not hcols:
        hcols = {}
    s = ""
    for i in range(n+1):
        s = s + ('|c' if i in hcols else 'c')
    #版头版尾
    beg="\\begin{align*}\n\\left(\\begin{array}{%s}\n"%s[:-1]
    end = "\n\\end{array}\\right)\n\\end{align*}"
    #内容部分
    content = "\\hline\n" if 0 in hrows else ""
    for i,line in enumerate(mat):
        for s in line[:-1]:
            content = content + s + ' &'
        content = content + line[-1] + '\\\\'
        content = content + ('\\hline\n' if i+1 in hrows else '\n')
    content = (content[:-9] if hrows[-1] else content[:-3])#去掉多余末尾
    out = beg+content+end
    print(out)
    pyperclip.copy(out)

In [ ]:
#过渡矩阵
f = {i+1:j for i,j in enumerate(fs)}
roots = [(3,4,2,2,1),(3,7,2,1),(4,6,2,1),(3,4,2,5),(10,2,1),(6,7,1),(3,2,9),(4,2,8),(3,7,5),(4,6,5),(10,5),(2,11),(6,9),(7,8),(12,)] #录入索引
roots = [tuple(f[i] for i in line) for line in roots] #转字符串
roots = [ide.tuple2element(i) for i in roots] #转pbw元素
keys = [list(i.keys)[0] for i in pbw_basis] #键值
mat = matrix([[root.element[key] if key in root else 0 for root in roots] for key in keys])

In [ ]:
print(mat)

In [ ]:
Lie.symbols('x',3)
#res = [1, - x2 - x3, - 2*x1 - 2, x3 - x2, 5*x1 + x2 + 2*x3 + x1*x2 + x1*x3 + x2*x3 + x1^2 + 4, - x1^2 - 3*x1 + x2^2 + x2 - 2, - x1^2 - 3*x1 + x3^2 - x2 - 2, x2 - 3*x1 + x1*x2 + x1*x3 - x2*x3 - x1^2 - 2, x2 - 3*x1 + x1*x2 - x1*x3 + x2*x3 - x1^2 - 2, 5*x1 + x2 - 2*x3 + x1*x2 - x1*x3 - x2*x3 + x1^2 + 4, 8*x1 - 2*x2 + 2*x3 - 2*x1*x2 + 4*x1*x3 - 2*x2*x3 - x1*x2^2 + x1^2*x3 - x2^2*x3 + 5*x1^2 + x1^3 - x2^2 + 4, x2*x3^2 - x2 - 2*x1*x2 - x1^2*x2 - x1*x3^2 - x1 + 2*x1^2 + x1^3 - 2, 13*x1 + 3*x2 + 4*x1*x2 + x1^2*x2 - x1*x3^2 - x2*x3^2 + 6*x1^2 + x1^3 - 2*x3^2 + 8, 8*x1 - 2*x2 - 2*x3 - 2*x1*x2 - 4*x1*x3 + 2*x2*x3 - x1*x2^2 - x1^2*x3 + x2^2*x3 + 5*x1^2 + x1^3 - x2^2 + 4, x1^4 + 7*x1^3 - x1^2*x2^2 - x1^2*x2 - x1^2*x3^2 + 19*x1^2 - 4*x1*x2^2 - 6*x1*x2 - 3*x1*x3^2 + 23*x1 + x2^2*x3^2 - 3*x2^2 + x2*x3^2 - 5*x2 - 2*x3^2 + 10]
res = matrix(res).T
res2 = mat.inverse()*res
res2 = [(i+1,factor(j)) for i,j in enumerate(res2.column(0))]
res = [(i+1,factor(j)) for i,j in enumerate(res.column(0))]

In [ ]:
table(res)

In [ ]:
table(res2)

In [ ]:
for i in range(15):
    a = Matrix([mat_sym.col(j).transpose() for j in range(15) if j!=i])
    print(a.rank())

In [ ]:
185+262+50+184+184+50+81+133

In [ ]:
'''计算时间
2阶：1秒左右 5*4
3阶：14秒左右 23*15
4阶：134秒左右 106*55
5阶：1116秒左右 462*200
''';

In [ ]:
#matlab矩阵命令
print(symmat_to_matlab(mat_sym,n))

In [ ]:
#matlab解方程组的命令
print(symmat_to_command(mat_sym,n))

In [ ]:
#matlab解信息的命令
print(read_solution(len(dec)-1))